# PRO LEVEL CHALLENGE 2: Clean Data > Big Models (NER Check)

**The Problem:** Simple keyword searches are "dumb." If you search for "Apple," you might get news about fruit or pie. If you search for "Amazon," you might get rainforest news.

**Your Mission:** Implement a Named Entity Recognition (NER) check using **spaCy**. Only keep articles where the entity type is **ORG** (Organization).

## Setup: Install spaCy and Download Model

In [1]:
# Install spaCy and download English model
# !pip install spacy
# !python -m spacy download en_core_web_sm

import spacy
import pandas as pd

# Load spaCy model
nlp = spacy.load('en_core_web_sm')
print("✅ spaCy model loaded successfully")

✅ spaCy model loaded successfully


## Understanding Named Entities

spaCy scans text and identifies "Entities." For financial data, we care about:
- **ORG**: Organizations (Companies, agencies)
- **GPE**: Geopolitical entities (Countries, cities)
- **PERSON**: People names
- **MONEY**: Monetary values

**Goal:** If "Apple" is labeled as a fruit (no label) or "Amazon" is labeled as a **LOC** (Location), we discard it.

In [3]:
# Example: Identify entities in sample texts
sample_texts = [
    "Apple Inc. reports record earnings in Q4",
    "The Amazon rainforest is facing deforestation",
    "Amazon Web Services expands cloud infrastructure",
    "I bought an apple from the grocery store",
    "Tesla opens new factory in Austin, Texas",
    "Visit the Tesla museum in New York"
]

print("Entity Detection Examples:")
print("=" * 60)

for text in sample_texts:
    doc = nlp(text)
    print(f"\nText: {text}")
    if doc.ents:
        for ent in doc.ents:
            print(f"  - '{ent.text}' → {ent.label_}")
    else:
        print("  - No entities found")

Entity Detection Examples:

Text: Apple Inc. reports record earnings in Q4
  - 'Apple Inc.' → ORG
  - 'Q4' → GPE

Text: The Amazon rainforest is facing deforestation
  - 'Amazon' → ORG

Text: Amazon Web Services expands cloud infrastructure
  - 'Amazon Web Services' → ORG

Text: I bought an apple from the grocery store
  - No entities found

Text: Tesla opens new factory in Austin, Texas
  - 'Austin' → GPE
  - 'Texas' → GPE

Text: Visit the Tesla museum in New York
  - 'Tesla' → NORP
  - 'New York' → GPE


> ### The "Ambiguity" Challenge (False Positives)
>
> Even with NER, some "noise" can slip through. For example, a basic model might still label the **Amazon Rainforest** or **Apple trees** as an **ORG** (Organization) because it sees these names used as companies 90% of the time in news data.

### Step 1: Create the NER Filter
We need a function that returns `True` only if our target company is found and labeled specifically as an **Organization (ORG)**.

In [10]:
def check_entity_is_org(text, company_name):
    """
    Check if the company_name in the text is classified as an ORG.
    """
    doc = nlp(text)

    for ent in doc.ents:
        # Check if entity text contains company name (case-insensitive)
        if company_name.lower() in ent.text.lower():
            if ent.label_ == 'ORG':
                return True
            else:
                return False  # Found but not an ORG (e.g., a Location)

    return False  # Company name not found as an entity at all

# Test the function
test_cases = [
    ("Apple Inc. releases new iPhone", "Apple", True),
    ("I ate an apple for lunch", "Apple", False),
    ("Amazon expands its e-commerce platform", "Amazon", True),
    ("The Amazon river is the longest", "Amazon", False),
]

print("Testing NER Filter:")
print("=" * 60)
for text, company, expected in test_cases:
    result = check_entity_is_org(text, company)
    status = "✅" if result == expected else "❌"
    print(f"{status} '{text}' → {result} (expected {expected})")

Testing NER Filter:
✅ 'Apple Inc. releases new iPhone' → True (expected True)
✅ 'I ate an apple for lunch' → False (expected False)
✅ 'Amazon expands its e-commerce platform' → True (expected True)
❌ 'The Amazon river is the longest' → True (expected False)


### Step 2: Cleaning the News Dataset
Now we apply this filter to a dataframe. This mimics your actual pipeline where you fetch raw headlines and need to prune the "noise."

In [14]:
# Create sample news dataset with mixed contexts
news_data = {
    'headline': [
        'Apple announces new product line',
        'How to make apple pie at home',
        'Amazon stock surges on earnings beat',
        'Amazon rainforest conservation efforts',
        'Tesla unveils new electric vehicle',
        'Visit the Tesla museum in Belgrade',
        'Microsoft launches AI initiative',
        'Google reports strong ad revenue',
        'Trip to New York and visiting Google offices',
        'Apple trees need proper care'
    ],
    'company': ['Apple', 'Apple', 'Amazon', 'Amazon', 'Tesla',
               'Tesla', 'Microsoft', 'Google', 'Google', 'Apple']
}

df = pd.DataFrame(news_data)

# Apply NER filter
df['is_org'] = df.apply(
    lambda row: check_entity_is_org(row['headline'], row['company']),
    axis=1
)

# Filter to keep only ORG entities
df_filtered = df[df['is_org'] == True].copy()

print(f"Filtered dataset: {len(df_filtered)} articles (removed {len(df) - len(df_filtered)} non-ORG)")
print("\nCleaned News for Analysis:")
print(df_filtered[['headline', 'company']])

Filtered dataset: 7 articles (removed 3 non-ORG)

Cleaned News for Analysis:
                                       headline    company
0              Apple announces new product line      Apple
2          Amazon stock surges on earnings beat     Amazon
3        Amazon rainforest conservation efforts     Amazon
6              Microsoft launches AI initiative  Microsoft
7              Google reports strong ad revenue     Google
8  Trip to New York and visiting Google offices     Google
9                  Apple trees need proper care      Apple


> ### The "Ambiguity" Challenge (False Positives)
>
> **How to fix this:** To reach 100% accuracy, professionals often use a "Double Check" method—combining NER with **negative keywords** (like "tree" or "rainforest") to discard remaining non-business articles.

In [7]:
def check_entity_is_org_refined(text, company_name):
    """
    Enhanced filter that uses both NER and keyword exclusion
    to remove non-business news.
    """
    # 1. Define 'Noise' keywords that don't belong in stock news
    noise_keywords = ['rainforest', 'river', 'tree', 'fruit', 'pie', 'recipe', 'care']

    # 2. Pre-filter: if any noise word exists, reject immediately
    if any(word in text.lower() for word in noise_keywords):
        return False

    doc = nlp(text)

    # 3. Standard NER check
    for ent in doc.ents:
        if company_name.lower() in ent.text.lower():
            # Only keep if strictly identified as an Organization
            return ent.label_ == 'ORG'

    return False

# Re-apply the refined filter to your DataFrame
df['is_org_refined'] = df.apply(
    lambda row: check_entity_is_org_refined(row['headline'], row['company']),
    axis=1
)

# Show the results
df_final = df[df['is_org_refined'] == True].copy()
print(f"✅ Final Cleaned Dataset: {len(df_final)} articles")
print(df_final[['headline', 'company']])

✅ Final Cleaned Dataset: 5 articles
                                       headline    company
0              Apple announces new product line      Apple
2          Amazon stock surges on earnings beat     Amazon
6              Microsoft launches AI initiative  Microsoft
7              Google reports strong ad revenue     Google
8  Trip to New York and visiting Google offices     Google


In [12]:
# Apply the Refined NER filter
df['is_org'] = df.apply(
    lambda row: check_entity_is_org_refined(row['headline'], row['company']),
    axis=1
)

print("Dataset with NER check:")
print(df)

# Filter to keep only ORG entities
df_filtered = df[df['is_org'] == True].copy()

print(f"✅ Filtered dataset: {len(df_filtered)} articles (removed {len(df) - len(df_filtered)} non-ORG)")
print("Filtered articles:")
print(df_filtered[['headline', 'company']])

Dataset with NER check:
                                       headline    company  is_org
0              Apple announces new product line      Apple    True
1                 How to make apple pie at home      Apple   False
2          Amazon stock surges on earnings beat     Amazon    True
3        Amazon rainforest conservation efforts     Amazon   False
4            Tesla unveils new electric vehicle      Tesla   False
5            Visit the Tesla museum in Belgrade      Tesla   False
6              Microsoft launches AI initiative  Microsoft    True
7              Google reports strong ad revenue     Google    True
8  Trip to New York and visiting Google offices     Google    True
9                  Apple trees need proper care      Apple   False
✅ Filtered dataset: 5 articles (removed 5 non-ORG)
Filtered articles:
                                       headline    company
0              Apple announces new product line      Apple
2          Amazon stock surges on earnings beat    

## Batch Processing for Large Datasets

### 🚀 Speeding Up with Batch Processing

When processing thousands of headlines, using `.apply()` can be slow because it loads the model for every single row.

**How `nlp.pipe` works:**
* It buffers headlines and processes them in parallel batches.
* It is significantly faster than standard loops.
* We integrated our **Noise Filter** directly into the batch loop to ensure data quality remains high.

In [13]:
# For large datasets, use spaCy's pipe for efficiency
def filter_org_articles_batch(df, text_column='headline', company_column='company'):
    """
    Efficiently filter articles using spaCy's pipe with noise reduction.
    """
    results = []
    noise_keywords = ['rainforest', 'river', 'tree', 'fruit', 'pie', 'recipe', 'care']

    # Process in batches using nlp.pipe for high performance
    docs = nlp.pipe(df[text_column])

    for idx, doc in enumerate(docs):
        headline_text = df.iloc[idx][text_column].lower()
        company = df.iloc[idx][company_column].lower()
        is_org = False

        # 1. First, check for noise keywords
        if any(word in headline_text for word in noise_keywords):
            results.append(False)
            continue

        # 2. Then, check the entities detected by spaCy
        for ent in doc.ents:
            if company in ent.text.lower() and ent.label_ == 'ORG':
                is_org = True
                break

        results.append(is_org)

    return results

# Test with our dataset
df['is_org_batch'] = filter_org_articles_batch(df)

print("Batch processing results:")
print(df[['headline', 'company', 'is_org_batch']])

Batch processing results:
                                       headline    company  is_org_batch
0              Apple announces new product line      Apple          True
1                 How to make apple pie at home      Apple         False
2          Amazon stock surges on earnings beat     Amazon          True
3        Amazon rainforest conservation efforts     Amazon         False
4            Tesla unveils new electric vehicle      Tesla         False
5            Visit the Tesla museum in Belgrade      Tesla         False
6              Microsoft launches AI initiative  Microsoft          True
7              Google reports strong ad revenue     Google          True
8  Trip to New York and visiting Google offices     Google          True
9                  Apple trees need proper care      Apple         False
